# New Direct Inversion in Iterative Space (DIIS)

This method has been extensively used to solve self-consistent field (SCF) problems in the fields of quantum chemistry and physics. In this tutorial, we employ this method to accelerate the solving of fixed-point problems.

It should be noted that one potential issue with this method is that non-negative parameters cannot be guaranteed during optimization in the conventional DIIS approach. Although this issue can be addressed by explicitly introducing constraints to the linear combination coefficients, key concepts in DIIS, numerical issues may still arise, such as singular matrices or convergence problems.

In [1]:
import logging

import numpy as np
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import (
    BasisFuncHelper,
    LinearISAWPart,
    check_pro_atom_parameters,
    compute_quantities,
    lstsq_solver_dyn,
    lstsq_solver_with_extra_constr,
    lstsq_spsolver,
)

In DIIS method, the combination coefficients are determined by solving a least-square problem.
The default method is `lstsq_spsolver`, which computes the inversion of the matrix directly.
One can also use other least-square solver and availiable methods are `lstsq_solver_dyn`, `lstsq_solver_with_extra_constr`.

In [2]:
def run_diis(diis_method):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "diis"
    print("*" * 80)
    print(f"Results with {diis_method.__name__}".center(80, " "))
    print("*" * 80)
    kwargs["solver_options"] = {"diis_size": 8, "lstsq_solver": diis_method}
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [3]:
run_diis(lstsq_spsolver)

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]
********************************************************************************
                          Results with lstsq_spsolver                           
********************************************************************************

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 3

In [4]:
def run_cdiis(**solver_options):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "cdiis"
    kwargs["solver_options"] = solver_options
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [5]:
run_cdiis(mode="R-CDIIS", param=1e-2)

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ At

In [6]:
run_cdiis(mode="FD-CDIIS", diis_size=5)

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ At

In [7]:
run_cdiis(mode="AD-CDIIS", param=1e-4)

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ At

In [8]:
run_cdiis(mode="Roothaan")

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ At